# 1) Titolo e obiettivi della lezione

Lezione 2 — Filtri e Feature Engineering con Pandas

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

1. Applicare **boolean indexing** per filtrare righe di un DataFrame
2. Combinare condizioni multiple con operatori logici (`&`, `|`, `~`)
3. Usare **`.loc[]`** per selezionare righe e colonne contemporaneamente
4. Creare **nuove feature** a partire da colonne esistenti (feature engineering)
5. Ordinare i risultati con **`.sort_values()`**
6. Riconoscere pattern ricorrenti nella manipolazione dati

---

## Perche' questi obiettivi contano

Questa struttura obbligatoria rende il notebook autonomo: ogni blocco spiega perche' il passo e' necessario, quali forme/NaN aspettarsi e come reagire se i controlli falliscono.

Il filtraggio e il feature engineering sono le operazioni piu frequenti nell'analisi dati. Ogni progetto di data science richiede di selezionare sottoinsiemi di dati e creare nuove variabili derivate. Senza queste competenze, non puoi procedere con nessuna analisi seria.

## Prerequisiti

- Lezione 1: Fondamenti di NumPy (array, broadcasting, boolean mask)
- Conoscenza base di Python (variabili, liste, dizionari)

---

## Indice (struttura a 8 sezioni)

1. Titolo e obiettivi della lezione
2. Teoria concettuale profonda (con glossario e formule spiegate)
3. Schema mentale / mappa decisionale
4. Sezione dimostrativa con checkpoint e sanity check
5. Esercizi risolti passo passo + errori comuni / debug rapido
6. Conclusione operativa
7. Checklist di fine lezione e micro-validazioni
8. Changelog didattico

## Librerie utilizzate

```python
import pandas as pd
import numpy as np
```

## Convenzioni di nomenclatura

- `df` per il DataFrame principale
- Maschere booleane con prefisso `mask_` o `maschera_`
- Feature derivate con nomi descrittivi: `Indice_Comfort`, `Molto_Secco`

---

# 2) Teoria concettuale profonda

## 2.1 Cos'e un DataFrame?

Un **DataFrame** e la struttura dati principale di Pandas: una tabella bidimensionale con:
- **Righe** identificate da un indice (numerico o etichetta)
- **Colonne** identificate da nomi (stringhe)
- Ogni colonna e una **Series** (array 1D con indice)

```
          Colonna1    Colonna2    Colonna3
Indice 0     A           10         True
Indice 1     B           20         False
Indice 2     C           30         True
```

**Differenza tra DataFrame e array NumPy:**

| Aspetto | Array NumPy | DataFrame Pandas |
|---------|-------------|------------------|
| Dimensioni | Qualsiasi (1D, 2D, nD) | Solo 2D (righe x colonne) |
| Etichette | Solo indici numerici | Nomi per colonne, etichette per righe |
| Tipi dati | Omogenei (tutti uguali) | Eterogenei (ogni colonna puo avere tipo diverso) |
| Uso tipico | Calcoli numerici | Dati tabulari (CSV, database) |

**Perche usare DataFrame invece di array NumPy?**

I DataFrame sono progettati per dati del mondo reale:
- Colonne con nomi descrittivi (`"Temperatura"` invece di colonna 0)
- Tipi misti (stringhe, numeri, date nella stessa tabella)
- Gestione nativa di valori mancanti (NaN)
- Operazioni di aggregazione per gruppo (groupby)

## 2.2 Boolean Indexing: Il Cuore del Filtraggio

Il **boolean indexing** e la tecnica per selezionare righe in base a condizioni logiche. E la stessa tecnica vista in NumPy, applicata ai DataFrame.

**Processo in due fasi:**

1. **Crei una maschera booleana**: `df["Colonna"] > valore` produce una Series di True/False
2. **Applichi la maschera**: `df[maschera]` restituisce solo le righe dove la maschera e True

**Esempio concettuale:**
```
DataFrame originale:
   Nome    Eta
0  Alice   25
1  Bob     35
2  Carol   42
3  David   28

df["Eta"] > 30  →  Series: [False, True, True, False]

df[df["Eta"] > 30]  →  Righe 1 e 2 (Bob e Carol)
```

**Perche il boolean indexing e cosi potente?**

1. **Espressivo**: scrivi condizioni leggibili (`df["Prezzo"] < 100`)
2. **Veloce**: opera su tutto il DataFrame senza cicli Python
3. **Componibile**: combini condizioni con operatori logici
4. **Flessibile**: funziona con qualsiasi tipo di confronto

## 2.3 Operatori Logici per Condizioni Multiple

Quando devi filtrare con piu di una condizione, usi gli operatori logici:

| Operatore | Significato | Esempio | Risultato True quando |
|-----------|-------------|---------|----------------------|
| `&` | AND | `(cond1) & (cond2)` | ENTRAMBE le condizioni sono True |
| `\|` | OR | `(cond1) \| (cond2)` | ALMENO UNA condizione e True |
| `~` | NOT | `~cond1` | La condizione e False |

**ATTENZIONE CRITICA**: Le parentesi sono **obbligatorie** attorno a ogni condizione!

```python
# CORRETTO
(df["A"] > 5) & (df["B"] < 10)

# SBAGLIATO - errore di precedenza operatori
df["A"] > 5 & df["B"] < 10
```

**Perche le parentesi sono obbligatorie?**

Gli operatori `&` e `|` hanno precedenza piu alta degli operatori di confronto (`>`, `<`, `==`). Senza parentesi, Python interpreta l'espressione in modo errato:

```python
# Senza parentesi, Python legge:
df["A"] > (5 & df["B"]) < 10  # Non quello che vuoi!

# Con parentesi, Python legge:
(df["A"] > 5) & (df["B"] < 10)  # Corretto!
```

## 2.4 `.loc[]` vs Boolean Indexing Semplice

Pandas offre due modi per filtrare:

| Sintassi | Cosa Fa | Quando Usarla |
|----------|---------|---------------|
| `df[maschera]` | Filtra righe, mantiene tutte le colonne | Solo filtraggio righe |
| `df.loc[maschera, colonne]` | Filtra righe E seleziona colonne | Filtraggio + selezione colonne |
| `df.loc[maschera, "Colonna"]` | Filtra righe, una sola colonna | Estrarre una colonna filtrata (restituisce Series) |

**Regola pratica**: Usa `.loc[]` quando devi specificare anche le colonne. Usa `df[mask]` per filtri semplici.

**Differenza tra .loc e .iloc:**

| Metodo | Basato su | Esempio |
|--------|-----------|---------|
| `.loc[]` | Etichette (nomi) | `df.loc[0:5, "Nome"]` |
| `.iloc[]` | Posizioni (numeri) | `df.iloc[0:5, 0]` |

## 2.5 Feature Engineering: Creare Nuove Variabili

Il **feature engineering** e il processo di creare nuove colonne derivate da quelle esistenti. E una delle attivita piu importanti nel machine learning perche:

1. **Cattura relazioni non lineari**: combinando variabili puoi catturare pattern che una singola variabile non mostra
2. **Codifica conoscenza del dominio**: trasformi intuizioni in variabili numeriche
3. **Migliora le performance dei modelli**: feature ben progettate possono migliorare drasticamente i risultati

**Tipi di feature engineering:**

| Tipo | Esempio | Quando usarlo |
|------|---------|---------------|
| **Aritmetica** | `df["Totale"] = df["Prezzo"] * df["Quantita"]` | Combinare variabili correlate |
| **Trasformazione** | `df["Log_Prezzo"] = np.log(df["Prezzo"])` | Ridurre skewness |
| **Booleana (Flag)** | `df["Alto"] = df["Altezza"] > 180` | Creare categorie binarie |
| **Categorizzazione** | `pd.cut(df["Eta"], bins=[0,18,65,100])` | Creare fasce |
| **Statistica** | `df["Z_Score"] = (df["X"] - df["X"].mean()) / df["X"].std()` | Normalizzazione |

**Formula dell'Indice di Comfort (esempio di questa lezione):**

$$\text{Indice\_Comfort} = \text{Temperatura} - \frac{\text{Umidita}}{8}$$

Questa formula bilancia temperatura e umidita: alta temperatura e bassa umidita danno comfort maggiore. Il divisore 8 e scelto empiricamente per bilanciare le scale delle due variabili.

## 2.6 Statistiche Descrittive per Soglie Dinamiche

Invece di usare valori fissi (es. "temperatura > 20"), usiamo statistiche per creare soglie che si adattano ai dati:

| Metodo | Significato | Uso Tipico | Vantaggio |
|--------|-------------|------------|-----------|
| `.mean()` | Media aritmetica | "Sopra/sotto la media" | Semplice e intuitiva |
| `.median()` | Valore centrale | Robusto agli outlier | Non distorta da valori estremi |
| `.quantile(0.25)` | Primo quartile (Q1) | "Nel 25% piu basso" | Definisce fasce di dati |
| `.quantile(0.75)` | Terzo quartile (Q3) | "Nel 25% piu alto" | Identifica valori elevati |
| `.std()` | Deviazione standard | Per z-score e anomalie | Misura dispersione |

**Perche usare soglie dinamiche invece di valori fissi?**

1. **Adattabilita**: il codice funziona anche se i dati cambiano
2. **Generalizzazione**: non devi conoscere i valori esatti in anticipo
3. **Riproducibilita**: la logica e chiara e documentata
4. **Robustezza**: eviti valori "magici" hardcoded

## 2.7 Ordinamento con `.sort_values()`

L'ordinamento permette di:
- Identificare i valori massimi/minimi
- Creare ranking
- Preparare i dati per visualizzazioni
- Trovare i "top N" o "bottom N"

```python
df.sort_values("Colonna", ascending=False)  # Decrescente (dal piu alto)
df.sort_values("Colonna", ascending=True)   # Crescente (dal piu basso, default)
df.sort_values(["Col1", "Col2"])            # Multi-colonna: prima per Col1, poi Col2
```

**Attenzione**: `.sort_values()` restituisce una **copia** del DataFrame. L'originale non viene modificato a meno che non usi `inplace=True`.

In [ ]:
# =============================================================================
# SETUP: Import librerie e creazione DataFrame
# =============================================================================
# Importiamo le librerie necessarie con gli alias standard.
# pandas (pd): per manipolazione dati tabulari (DataFrame)
# numpy (np): per operazioni numeriche (usato per trasformazioni)

import pandas as pd   # Libreria per manipolazione dati tabulari
import numpy as np    # Libreria per operazioni numeriche

# Creiamo un DataFrame con dati meteorologici settimanali.
# pd.DataFrame() accetta un dizionario dove:
# - chiavi = nomi delle colonne
# - valori = liste di dati (tutti della stessa lunghezza)

df = pd.DataFrame({
    "Giorno": ["Lun", "Mar", "Mer", "Gio", "Ven", "Sab", "Dom"],
    "Temperatura": [18, 20, 21, 19, 23, 25, 22],  # Gradi Celsius
    "Umidita": [60, 55, 58, 63, 50, 45, 52],      # Percentuale
    "Pioggia": [0, 1, 0, 1, 0, 0, 1]              # 0=No, 1=Si
})

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifichiamo che il DataFrame sia stato creato correttamente
# -----------------------------------------------------------------------------
print("=== VERIFICA CREAZIONE DATAFRAME ===")
print(f"Tipo oggetto: {type(df).__name__}")
print(f"Dimensioni: {df.shape[0]} righe x {df.shape[1]} colonne")
print(f"Colonne: {list(df.columns)}")
print(f"Tipi di dato per colonna:")
print(df.dtypes)

# Sanity check: verifichiamo che tutte le colonne abbiano la stessa lunghezza
assert df.shape[0] == 7, f"ERRORE: attese 7 righe, trovate {df.shape[0]}"
assert df.shape[1] == 4, f"ERRORE: attese 4 colonne, trovate {df.shape[1]}"
print("\nSanity check superato: DataFrame creato correttamente!")

# Visualizziamo il DataFrame
print("\n=== CONTENUTO DATAFRAME ===")
df

,Giorno,Temperatura,Umidita,Pioggia
0,Lun,18,60,0
1,Mar,20,55,1
2,Mer,21,58,0
3,Gio,19,63,1
4,Ven,23,50,0
5,Sab,25,45,0
6,Dom,22,52,1


---

# 3) Schema mentale / mappa decisionale

## Quando Usare Quale Tecnica di Filtraggio?

```
DEVO SELEZIONARE DATI DA UN DATAFRAME
                |
                v
    Ho bisogno di filtrare RIGHE?
           /            \
         SI              NO
          |               |
          v               v
  Creo maschera      Accedo direttamente
  booleana           df["Colonna"]
          |
          v
    Quante condizioni?
       /        \
     UNA      MULTIPLE
      |           |
      v           v
  df[cond]   Combino con & | ~
      |           |
      v           v
    Devo anche selezionare COLONNE specifiche?
           /            \
         SI              NO
          |               |
          v               v
   df.loc[mask, cols]  df[mask]
```

## Albero decisionale: Feature Engineering

```
DEVO CREARE UNA NUOVA FEATURE?
                |
                v
    Che tipo di feature?
    /      |       \      \
ARITMETICA BOOLEANA CATEGORICA STATISTICA
    |         |         |          |
    v         v         v          v
 df["A"]+   df["A"]>  pd.cut()   df["A"]-
 df["B"]    soglia    pd.qcut()  df["A"].mean()
    |         |         |          |
    v         v         v          v
 Nuova     Flag        Fasce     Z-score o
 colonna   True/False  nominali  normalizzato
```

## Scelta della statistica per la soglia

| Domanda | Statistica | Perche |
|---------|------------|--------|
| Voglio dividere a meta? | `.median()` | Esattamente 50% sopra e sotto |
| Voglio la soglia "tipica"? | `.mean()` | Considera tutti i valori |
| Dati con outlier? | `.median()` | Robusta agli estremi |
| Voglio il 25% piu basso? | `.quantile(0.25)` | Definisce Q1 |
| Voglio il 25% piu alto? | `.quantile(0.75)` | Definisce Q3 |

## Checklist Pre-Filtraggio

Prima di applicare un filtro, verifica:

1. **Quale colonna devo confrontare?** → Verifica che esista: `col in df.columns`
2. **Con quale valore/soglia?** → Fisso (20) o calcolato (df["X"].mean())?
3. **Devo combinare piu condizioni?** → Prepara le parentesi: `(cond1) & (cond2)`
4. **Mi servono tutte le colonne o solo alcune?** → `df[mask]` vs `df.loc[mask, cols]`
5. **Devo ordinare i risultati?** → Aggiungi `.sort_values()` alla fine

## Pattern anti-errore: ordine delle operazioni

```python
# 1. Calcola le soglie PRIMA
media = df["X"].mean()
q1 = df["X"].quantile(0.25)

# 2. Crea le maschere SEPARATE (per debug)
mask1 = df["X"] > media
mask2 = df["Y"] < q1

# 3. Combina le maschere
mask_finale = mask1 & mask2

# 4. Applica il filtro
risultato = df[mask_finale]

# 5. Verifica con assert
assert len(risultato) >= 0, "Filtro fallito"
```

---

# 4) Sezione dimostrativa

## 4.1 Setup: Import e Creazione DataFrame

### Perche questo passaggio e necessario

Prima di poter filtrare o manipolare dati, dobbiamo avere un DataFrame su cui lavorare. In progetti reali caricheresti dati da file CSV, database o API. In questa lezione creiamo dati sintetici per avere controllo completo e poter verificare i risultati.

Il DataFrame che creiamo simula dati meteorologici settimanali con:
- **Giorno**: etichetta del giorno (categorica)
- **Temperatura**: gradi Celsius (numerica continua)
- **Umidita**: percentuale (numerica continua)
- **Pioggia**: flag binario 0/1 (numerica binaria)

Questo mix di tipi e rappresentativo dei dataset reali.

---

## 4.2 Filtri Multipli con Boolean Indexing

### Perche questo passaggio e necessario

Il filtraggio con condizioni multiple e l'operazione piu comune nell'analisi dati. Quasi mai filtriamo con una sola condizione: vogliamo trovare record che soddisfano TUTTE le condizioni (AND) o ALMENO UNA (OR).

In questo esempio, vogliamo trovare i giorni "ideali": temperatura sopra la media E senza pioggia. Questo richiede combinare due maschere booleane con l'operatore `&`.

**Cosa succederebbe se saltassimo questo passaggio?**
- Non sapresti combinare condizioni
- Saresti limitato a filtri banali con una sola condizione
- Non potresti fare analisi complesse sui dati

### Spiegazione del processo

1. **Calcolo la soglia**: `df["Temperatura"].mean()` → un numero
2. **Creo maschera 1**: `df["Temperatura"] > soglia` → Series di True/False
3. **Creo maschera 2**: `df["Pioggia"] == 0` → Series di True/False
4. **Combino con &**: `mask1 & mask2` → True solo dove ENTRAMBE sono True
5. **Applico il filtro**: `df[mask_combinata]` → solo righe True

### Concetto chiave: Precedenza degli operatori

Gli operatori `&` e `|` hanno precedenza MAGGIORE di `>`, `<`, `==`. Questo significa che Python valuta prima `&` e poi i confronti. Le parentesi forzano l'ordine corretto:

```python
# SENZA parentesi (SBAGLIATO):
df["A"] > 5 & df["B"] < 10
# Python legge: df["A"] > (5 & df["B"]) < 10 → ERRORE

# CON parentesi (CORRETTO):
(df["A"] > 5) & (df["B"] < 10)
# Python legge: (confronto1) & (confronto2) → OK
```

In [ ]:
# =============================================================================
# FILTRO 1: Temperatura sopra media E senza pioggia
# =============================================================================
# Obiettivo: trovare i giorni "ideali" per attivita all'aperto.
# Condizioni:
#   1. Temperatura > media (giornata calda per la settimana)
#   2. Pioggia == 0 (nessuna precipitazione)

# Passo 1: Calcoliamo prima la media per capire la soglia
# .mean() restituisce un singolo numero (scalare)
media_temp = df["Temperatura"].mean()
print("=== CALCOLO SOGLIA ===")
print(f"Media temperatura: {media_temp:.2f} gradi")
print(f"Tipo: {type(media_temp).__name__}")

# Passo 2: Costruiamo le due maschere booleane SEPARATAMENTE
# Questo aiuta il debug: possiamo ispezionare ogni maschera
mask_temp_alta = df["Temperatura"] > media_temp
mask_no_pioggia = df["Pioggia"] == 0

print(f"\n=== MASCHERE BOOLEANE ===")
print(f"Condizione: Temperatura > {media_temp:.2f}")
print(f"Maschera temp alta: {list(mask_temp_alta)}")
print(f"Condizione: Pioggia == 0")
print(f"Maschera no pioggia: {list(mask_no_pioggia)}")

# Passo 3: Combiniamo le maschere con AND (&)
# Il risultato e True SOLO dove ENTRAMBE le maschere sono True
mask_combinata = mask_temp_alta & mask_no_pioggia
print(f"\nMaschera combinata (AND): {list(mask_combinata)}")
print(f"Numero di True: {mask_combinata.sum()}")

# Passo 4: Applichiamo il filtro al DataFrame
# df[mask] restituisce un NUOVO DataFrame con solo le righe True
df_temp_alta_no_pioggia = df[mask_combinata]

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifichiamo il risultato
# -----------------------------------------------------------------------------
print(f"\n=== MICRO-CHECKPOINT ===")
print(f"Righe nel DataFrame originale: {len(df)}")
print(f"Righe dopo il filtro: {len(df_temp_alta_no_pioggia)}")

# Sanity check: il numero di righe deve corrispondere ai True nella maschera
assert len(df_temp_alta_no_pioggia) == mask_combinata.sum(), \
    "ERRORE: numero righe non corrisponde ai True!"

# Sanity check: tutte le righe filtrate devono soddisfare le condizioni
if len(df_temp_alta_no_pioggia) > 0:
    assert (df_temp_alta_no_pioggia["Temperatura"] > media_temp).all(), \
        "ERRORE: alcune righe hanno temperatura <= media!"
    assert (df_temp_alta_no_pioggia["Pioggia"] == 0).all(), \
        "ERRORE: alcune righe hanno pioggia!"
    print("Sanity check superato: tutte le righe soddisfano le condizioni!")

print("\n=== RISULTATO ===")
df_temp_alta_no_pioggia

,Giorno,Temperatura,Umidita,Pioggia
4,Ven,23,50,0
5,Sab,25,45,0


In [ ]:
# =============================================================================
# FILTRO con .loc[]: Selezionare righe E colonne
# =============================================================================
# A volte non vogliamo tutte le colonne, ma solo alcune.
# .loc[maschera, lista_colonne] permette di fare entrambe le operazioni.
#
# Sintassi: df.loc[condizione_righe, selezione_colonne]
# - condizione_righe: maschera booleana o slice
# - selezione_colonne: lista di nomi colonna, singolo nome, o slice

print("=== USO DI .loc[] PER RIGHE E COLONNE ===")

# Stesso filtro di prima, ma vogliamo solo la colonna "Giorno"
# Nota: la condizione e scritta direttamente dentro .loc[]
df_solo_giorni = df.loc[
    (df["Temperatura"] > df["Temperatura"].mean()) & (df["Pioggia"] == 0),
    ["Giorno"]  # Lista delle colonne da mantenere
]

print(f"Colonne selezionate: ['Giorno']")
print(f"Tipo risultato: {type(df_solo_giorni).__name__}")
print(f"Numero righe: {len(df_solo_giorni)}")

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifichiamo il tipo di risultato
# -----------------------------------------------------------------------------
# Con una lista ["Giorno"], il risultato e un DataFrame
# Con una stringa "Giorno", il risultato sarebbe una Series

assert isinstance(df_solo_giorni, pd.DataFrame), \
    "ERRORE: con lista di colonne dovrebbe restituire DataFrame!"
assert list(df_solo_giorni.columns) == ["Giorno"], \
    "ERRORE: dovrebbe avere solo la colonna Giorno!"

print("\n--- Micro-checkpoint superato: DataFrame con colonna singola ---")

# Confronto: cosa succede con stringa invece di lista?
print("\n=== DIFFERENZA TRA LISTA E STRINGA ===")
series_giorni = df.loc[
    (df["Temperatura"] > df["Temperatura"].mean()) & (df["Pioggia"] == 0),
    "Giorno"  # Stringa singola, non lista!
]
print(f"Con lista ['Giorno']: tipo = {type(df_solo_giorni).__name__}")
print(f"Con stringa 'Giorno': tipo = {type(series_giorni).__name__}")

df_solo_giorni

,Giorno
4,Ven
5,Sab


### 🎯 4.3 Filtraggio basato su statistiche calcolate: il Quantile

Finora abbiamo usato valori fissi (es. `> 20`) o statistiche semplici (es. `> media`).  
Ma il **Data Scientist professionista** usa spesso i **quantili** per creare filtri più robusti.

---

#### Perché i quantili sono superiori ai valori fissi?

1. **Adattabilità ai dati**: il valore cambia automaticamente se cambiano i dati
2. **Robustezza agli outlier**: la mediana (q=0.5) è meno influenzata dai valori estremi
3. **Interpretabilità statistica**: "il 25% più basso" è più significativo di "sotto 15"

---

#### I quantili fondamentali da conoscere

| Nome | Quantile | Significato |
|------|----------|-------------|
| Minimo | 0.0 | Il valore più piccolo |
| Primo quartile (Q1) | 0.25 | Il 25% dei dati è sotto questo valore |
| Mediana (Q2) | 0.5 | Il 50% dei dati è sotto questo valore |
| Terzo quartile (Q3) | 0.75 | Il 75% dei dati è sotto questo valore |
| Massimo | 1.0 | Il valore più grande |

---

#### Pattern comune: selezione della "coda" superiore

```python
soglia = df["Colonna"].quantile(0.75)   # Calcola il terzo quartile
df[df["Colonna"] > soglia]              # Seleziona il 25% più alto
```

---

#### ⚠️ Attenzione alla differenza: `>` vs `>=`

- `df["Temp"] > soglia` → **esclude** le righe esattamente uguali alla soglia
- `df["Temp"] >= soglia` → **include** le righe esattamente uguali alla soglia

Per le code (top 25%), generalmente si usa `>`.  
Per i percentili inclusivi (top 25% incluso il punto di taglio), si usa `>=`.

In [ ]:
# =============================================================================
# FILTRO CON QUANTILE: Selezionare le righe più calde
# =============================================================================
# Obiettivo: trovare le temperature nel quartile superiore (top 25%)
# Questo approccio è usato spesso per:
# - Identificare valori anomali (es. transazioni sospette)
# - Segmentare clienti (es. high-value customers)
# - Analisi di performance (es. prodotti best-seller)

print("=== FILTRO CON QUANTILE (Top 25% Temperatura) ===\n")

# Step 1: Calcolare la soglia (terzo quartile)
soglia_75 = df["Temperatura"].quantile(0.75)
print(f"Soglia Q3 (75° percentile): {soglia_75}°C")
print(f"Interpretazione: il 75% delle temperature è sotto {soglia_75}°C")

# Step 2: Creare la maschera booleana
maschera_top25 = df["Temperatura"] > soglia_75
print(f"\nMaschera top 25%: {maschera_top25.sum()} righe True su {len(maschera_top25)}")

# Step 3: Applicare il filtro
df_top_25_temp = df[maschera_top25]

# -----------------------------------------------------------------------------
# SANITY CHECK: Verifichiamo che le righe selezionate rispettino il criterio
# -----------------------------------------------------------------------------
print("\n--- Sanity Check ---")
temp_minima_selezionata = df_top_25_temp["Temperatura"].min()
print(f"Temperatura minima nel subset: {temp_minima_selezionata}°C")
print(f"È maggiore della soglia ({soglia_75}°C)? {temp_minima_selezionata > soglia_75}")

assert (df_top_25_temp["Temperatura"] > soglia_75).all(), \
    "ERRORE: qualche riga ha temperatura <= soglia!"

print("✓ Tutte le righe hanno Temperatura > soglia")

# Statistiche comparative
print(f"\n--- Statistiche comparative ---")
print(f"Media globale:     {df['Temperatura'].mean():.1f}°C")
print(f"Media top 25%:     {df_top_25_temp['Temperatura'].mean():.1f}°C")
print(f"Differenza:        +{df_top_25_temp['Temperatura'].mean() - df['Temperatura'].mean():.1f}°C")

df_top_25_temp

,Giorno,Temperatura
4,Ven,23
5,Sab,25
6,Dom,22


## 📌 Sezione 4.4 – Feature Engineering: Creare nuove colonne

Il **Feature Engineering** è l'arte di creare nuove colonne (features) a partire da quelle esistenti.  
È considerata una delle skill più importanti nel Machine Learning perché:

- I modelli non "capiscono" i dati grezzi: servono features significative
- Una buona feature può valere più di un modello complesso
- Spesso le relazioni interessanti sono **nascoste** nelle combinazioni di colonne

---

### Tipologie di Feature Engineering

| Tipo | Esempio | Quando usarlo |
|------|---------|---------------|
| **Aritmetica** | `colA + colB`, `colA * colB` | Per combinare grandezze correlate |
| **Trasformazioni** | `np.log(col)`, `col ** 2` | Per normalizzare distribuzioni skewed |
| **Indicatori binari** | `(col > soglia).astype(int)` | Per flag on/off |
| **Categorizzazione** | `pd.cut()`, `pd.qcut()` | Per discretizzare valori continui |
| **Aggregazioni** | `groupby().transform()` | Per statistiche per gruppo |

---

### Sintassi fondamentale: creazione colonna

```python
# Nuova colonna = formula con colonne esistenti
df["NuovaColonna"] = df["ColonnaA"] + df["ColonnaB"]

# Oppure con operazioni più complesse
df["NuovaColonna"] = (df["ColonnaA"] - df["ColonnaB"]).abs()
```

---

### ⚡ Best Practice: nomenclatura delle colonne

| Convenzione | Esempio | Vantaggio |
|-------------|---------|-----------|
| snake_case | `indice_comfort` | Standard Python, facile da digitare |
| Prefisso tipo | `is_caldo`, `has_pioggia` | Chiaro che è un booleano |
| Prefisso fonte | `raw_temp`, `calc_temp` | Distingue dati grezzi da calcolati |

---

### Nelle prossime celle creeremo:
1. **Indice di Comfort**: combinazione di temperatura e umidità
2. **Flag Giornata Calda**: indicatore binario per giorni caldi

In [ ]:
# =============================================================================
# FEATURE ENGINEERING: Creare l'Indice di Comfort
# =============================================================================
# L'Indice di Comfort è una misura sintetica che combina:
# - Temperatura: valori alti → comfort elevato (se non eccessivo)
# - Umidità: valori bassi → comfort elevato (aria meno "pesante")
#
# Formula scelta: IndiceComfort = Temperatura - (Umidità / 2)
# - Logica: la temperatura contribuisce positivamente
#           l'umidità viene "penalizzata" (divisa per 2 per bilanciare le scale)
#
# Nota: questa è una formula didattica. In metereologia esistono indici
# standardizzati come l'Heat Index o il Wet Bulb Globe Temperature (WBGT)

print("=== CREAZIONE INDICE DI COMFORT ===\n")

# Visualizza lo stato PRIMA della modifica
print(f"Colonne PRIMA: {list(df.columns)}")

# Creazione della nuova colonna
# Sintassi: df["NuovaColonna"] = formula
df["IndiceComfort"] = df["Temperatura"] - (df["Umidita"] / 2)

# Visualizza lo stato DOPO la modifica
print(f"Colonne DOPO:  {list(df.columns)}")

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifica della nuova colonna
# -----------------------------------------------------------------------------
assert "IndiceComfort" in df.columns, "ERRORE: colonna non creata!"
assert df["IndiceComfort"].notna().all(), "ERRORE: valori NaN presenti!"

print("\n--- Micro-checkpoint superato: colonna creata correttamente ---")

# Analisi della nuova feature
print(f"\n=== Statistiche IndiceComfort ===")
print(f"Min:     {df['IndiceComfort'].min():.1f}")
print(f"Max:     {df['IndiceComfort'].max():.1f}")
print(f"Media:   {df['IndiceComfort'].mean():.1f}")
print(f"Std:     {df['IndiceComfort'].std():.1f}")

# Verifica manuale su una riga
print(f"\n=== Verifica calcolo su prima riga ===")
temp_0 = df["Temperatura"].iloc[0]
umid_0 = df["Umidita"].iloc[0]
comfort_0 = df["IndiceComfort"].iloc[0]
calcolo_manuale = temp_0 - (umid_0 / 2)
print(f"Temperatura[0] = {temp_0}°C")
print(f"Umidità[0]     = {umid_0}%")
print(f"Formula: {temp_0} - ({umid_0}/2) = {calcolo_manuale:.1f}")
print(f"IndiceComfort[0] = {comfort_0:.1f}")
print(f"Match: {'✓' if abs(comfort_0 - calcolo_manuale) < 0.01 else '✗'}")

df[["Giorno", "Temperatura", "Umidita", "IndiceComfort"]].head()

,Giorno,Temperatura,Umidita,Pioggia,Indice_Comfort
0,Lun,18,60,0,10.500
1,Mar,20,55,1,13.125
2,Mer,21,58,0,13.750
3,Gio,19,63,1,11.125
4,Ven,23,50,0,16.750
5,Sab,25,45,0,19.375
6,Dom,22,52,1,15.500


In [ ]:
# =============================================================================
# FEATURE ENGINEERING: Creare un Flag Binario (Giornata Calda)
# =============================================================================
# Un "flag" è una colonna binaria (0/1 o True/False) che indica
# se una condizione è verificata oppure no.
#
# Uso tipico dei flag:
# - Segmentazione: dividere dati in gruppi
# - Filtraggio veloce: usare il flag come maschera
# - Machine Learning: feature categorica binaria
#
# Creeremo: GiornataCaldo = 1 se Temperatura > 25, altrimenti 0

print("=== CREAZIONE FLAG 'GiornataCaldo' ===\n")

# Definiamo la soglia (potrebbe venire da analisi precedente)
SOGLIA_CALDO = 25

# Metodo 1: Condizione booleana → produce True/False
maschera_caldo = df["Temperatura"] > SOGLIA_CALDO
print(f"Maschera booleana (True/False):\n{maschera_caldo.head()}\n")

# Metodo 2: Conversione in intero → produce 0/1
# .astype(int) converte True→1, False→0
df["GiornataCaldo"] = (df["Temperatura"] > SOGLIA_CALDO).astype(int)
print(f"Flag intero (0/1):\n{df['GiornataCaldo'].head()}\n")

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifica coerenza tra maschera e flag
# -----------------------------------------------------------------------------
assert (df["GiornataCaldo"] == maschera_caldo.astype(int)).all(), \
    "ERRORE: flag non coerente con maschera!"
print("--- Micro-checkpoint superato: flag coerente con maschera ---")

# Analisi della distribuzione
n_calde = df["GiornataCaldo"].sum()  # sum() di 0/1 = conta gli 1
n_totali = len(df)
perc_calde = 100 * n_calde / n_totali

print(f"\n=== Distribuzione del flag ===")
print(f"Giornate calde (>25°C):     {n_calde}/{n_totali} ({perc_calde:.1f}%)")
print(f"Giornate non calde (<=25°C): {n_totali - n_calde}/{n_totali} ({100 - perc_calde:.1f}%)")

# Verifica con value_counts()
print(f"\n=== Usando .value_counts() ===")
print(df["GiornataCaldo"].value_counts().sort_index())

# Confronto temperature medie per gruppo
print(f"\n=== Temperature medie per gruppo ===")
print(f"Media quando GiornataCaldo=0: {df[df['GiornataCaldo']==0]['Temperatura'].mean():.1f}°C")
print(f"Media quando GiornataCaldo=1: {df[df['GiornataCaldo']==1]['Temperatura'].mean():.1f}°C")

df[["Giorno", "Temperatura", "GiornataCaldo"]].head(7)

,Giorno,Temperatura,Umidita,Pioggia,Indice_Comfort,Molto_Secco
0,Lun,18,60,0,10.500,False
1,Mar,20,55,1,13.125,False
2,Mer,21,58,0,13.750,False
3,Gio,19,63,1,11.125,False
4,Ven,23,50,0,16.750,True
5,Sab,25,45,0,19.375,True
6,Dom,22,52,1,15.500,False


### 🎯 4.5 Ordinamento con `.sort_values()`: Ranking dei dati

Oltre al filtraggio e alla creazione di nuove colonne, un'operazione fondamentale  
è l'**ordinamento** dei dati secondo una o più colonne.

---

#### Perché ordinare i dati?

| Caso d'uso | Esempio |
|------------|---------|
| **Ranking** | Trovare i top-10 prodotti più venduti |
| **Analisi temporale** | Ordinare eventi per data |
| **Debugging** | Controllare valori estremi (min/max) |
| **Reportistica** | Presentare dati in ordine logico |

---

#### Sintassi di `.sort_values()`

```python
# Ordinamento singola colonna (ascendente)
df.sort_values(by="Colonna")

# Ordinamento discendente
df.sort_values(by="Colonna", ascending=False)

# Ordinamento multi-colonna
df.sort_values(by=["Colonna1", "Colonna2"], ascending=[True, False])
```

---

#### 🔑 Parametri chiave

| Parametro | Default | Descrizione |
|-----------|---------|-------------|
| `by` | Richiesto | Nome colonna o lista di colonne |
| `ascending` | True | True=crescente, False=decrescente |
| `inplace` | False | Se True, modifica il DataFrame originale |
| `na_position` | 'last' | Dove mettere i NaN: 'first' o 'last' |
| `ignore_index` | False | Se True, resetta l'indice 0,1,2,... |

---

#### ⚠️ Attenzione: `inplace=True` vs assegnazione

```python
# Metodo 1: con assegnazione (RACCOMANDATO)
df_ordinato = df.sort_values(by="Temp")

# Metodo 2: con inplace (modifica originale, RISCHIOSO)
df.sort_values(by="Temp", inplace=True)
```

Il metodo 1 è preferito perché:
- Preserva il DataFrame originale
- È più esplicito
- Evita bug difficili da trovare

In [ ]:
# =============================================================================
# ORDINAMENTO: Ranking delle giornate per Indice di Comfort
# =============================================================================
# Ordiniamo il DataFrame per IndiceComfort in ordine DECRESCENTE
# per trovare le giornate più confortevoli.
#
# Nota: usiamo ascending=False per avere i valori più alti in cima

print("=== ORDINAMENTO PER INDICE DI COMFORT ===\n")

# Ordinamento discendente (valori più alti prima)
df_ordinato = df.sort_values(by="IndiceComfort", ascending=False)

print("Top 5 giornate più confortevoli:")
print(df_ordinato[["Giorno", "Temperatura", "Umidita", "IndiceComfort"]].head())

# -----------------------------------------------------------------------------
# SANITY CHECK: Verifichiamo che l'ordinamento sia corretto
# -----------------------------------------------------------------------------
print("\n--- Sanity Check ---")
valori_comfort = df_ordinato["IndiceComfort"].values
is_decrescente = all(valori_comfort[i] >= valori_comfort[i+1] 
                     for i in range(len(valori_comfort)-1))
print(f"Ordinamento decrescente verificato: {'✓' if is_decrescente else '✗'}")

# Il primo valore deve essere il massimo
max_comfort = df["IndiceComfort"].max()
primo_valore = df_ordinato["IndiceComfort"].iloc[0]
print(f"Primo valore ({primo_valore:.1f}) = Massimo ({max_comfort:.1f})? {'✓' if primo_valore == max_comfort else '✗'}")

# Bonus: ordinamento multi-colonna
print("\n=== ORDINAMENTO MULTI-COLONNA ===")
print("Ordiniamo prima per GiornataCaldo (desc), poi per Temperatura (desc)")

df_multi_ord = df.sort_values(
    by=["GiornataCaldo", "Temperatura"], 
    ascending=[False, False]  # Entrambi decrescenti
)

print("\nRisultato: prima le giornate calde, poi ordinate per temperatura:")
print(df_multi_ord[["Giorno", "GiornataCaldo", "Temperatura"]].head(7))

# Osserva: le righe con GiornataCaldo=1 vengono prima,
# all'interno di ogni gruppo, ordinate per Temperatura decrescente

,Giorno,Temperatura,Umidita,Pioggia,Indice_Comfort,Molto_Secco
5,Sab,25,45,0,19.375,True
4,Ven,23,50,0,16.750,True
6,Dom,22,52,1,15.500,False
2,Mer,21,58,0,13.750,False
1,Mar,20,55,1,13.125,False
3,Gio,19,63,1,11.125,False
0,Lun,18,60,0,10.500,False


## 📌 Sezione 5 – Esercizi Pratici e Sfide

Ora che hai appreso le tecniche di filtraggio, feature engineering e ordinamento,  
è il momento di consolidare le competenze con esercizi mirati.

---

### Esercizio 5.1 – Filtri combinati

**Obiettivo**: Selezionare le giornate con:
- Temperatura tra 20°C e 28°C (estremi inclusi)
- Umidità inferiore al 60%

```python
# Hint: usa & per combinare le condizioni
# Ricorda le parentesi attorno a ogni condizione!
df_risultato = df[
    (df["Temperatura"] >= 20) & 
    (df["Temperatura"] <= 28) & 
    (df["Umidita"] < 60)
]
```

---

### Esercizio 5.2 – Feature Engineering avanzato

**Obiettivo**: Creare una colonna "CategoriaClimatica" che assume:
- "Freddo" se Temperatura < 15
- "Mite" se 15 <= Temperatura < 25
- "Caldo" se Temperatura >= 25

```python
# Hint: usa np.where() o pd.cut()
# Con np.where() annidato:
df["CategoriaClimatica"] = np.where(
    df["Temperatura"] < 15, "Freddo",
    np.where(df["Temperatura"] < 25, "Mite", "Caldo")
)
```

---

### Esercizio 5.3 – Analisi con ordinamento

**Obiettivo**: 
1. Creare un DataFrame con solo le giornate piovose (Pioggia == 1)
2. Ordinarlo per Umidità decrescente
3. Mostrare le prime 3 righe

---

### 🏆 Sfida bonus

Calcola la **correlazione** tra IndiceComfort e GiornataCaldo usando:
```python
df["IndiceComfort"].corr(df["GiornataCaldo"])
```

Che valore ti aspetti? Positivo o negativo? Perché?

## 📌 Sezione 6 – Glossario e Debug Reference

### 📖 Glossario dei termini chiave

| Termine | Definizione |
|---------|-------------|
| **Boolean Indexing** | Tecnica per selezionare righe usando una maschera di True/False |
| **Maschera booleana** | Series di valori True/False della stessa lunghezza del DataFrame |
| **Feature Engineering** | Processo di creazione di nuove colonne/features dai dati esistenti |
| **Flag** | Colonna binaria (0/1) che indica se una condizione è verificata |
| **Quantile** | Valore che divide i dati in parti uguali (es. mediana = quantile 0.5) |
| **Percentile** | Quantile espresso in percentuale (es. 75° percentile = quantile 0.75) |
| **Filtering** | Operazione di selezione di un sottoinsieme di righe |
| **.loc[]** | Accessor per selezione basata su label (righe E colonne) |
| **.iloc[]** | Accessor per selezione basata su posizione intera |
| **Broadcast** | Applicazione automatica di operazioni elemento per elemento |

---

### 🐛 Errori comuni e soluzioni

| Errore | Causa | Soluzione |
|--------|-------|-----------|
| `TypeError: cannot compare` | Confronto tra tipi incompatibili | Verificare i dtypes con `.dtypes` |
| `ValueError: shape mismatch` | Maschera di lunghezza diversa | Usare maschera derivata dallo stesso df |
| `KeyError: 'Colonna'` | Nome colonna errato | Controllare `df.columns` |
| `SettingWithCopyWarning` | Modifica su view anziché copy | Usare `.copy()` o `.loc[]` esplicito |
| Parentesi mancanti | Operatori `&` `|` hanno precedenza alta | Sempre parentesi: `(cond1) & (cond2)` |

---

### 💡 Pattern di debug consigliati

```python
# 1. Verificare la maschera prima di applicarla
maschera = df["Col"] > valore
print(f"True: {maschera.sum()}, False: {(~maschera).sum()}")

# 2. Controllare i tipi
print(df.dtypes)

# 3. Campione dei dati
print(df.head(3))

# 4. Forma del risultato
risultato = df[maschera]
print(f"Righe originali: {len(df)}, Righe filtrate: {len(risultato)}")
```

### ⚠️ Approfondimento: Errori specifici del Boolean Indexing

#### Errore 1: Uso di `and`/`or` invece di `&`/`|`

```python
# ❌ SBAGLIATO: ValueError
df[(df["A"] > 0) and (df["B"] > 0)]

# ✓ CORRETTO: usa operatori bitwise
df[(df["A"] > 0) & (df["B"] > 0)]
```

**Spiegazione**: `and`/`or` sono operatori Python per singoli booleani,  
`&`/`|` sono operatori per arrays/Series di booleani.

---

#### Errore 2: Maschera non allineata

```python
# ❌ SBAGLIATO: maschera da altro DataFrame
maschera_altro = altro_df["Col"] > 0
df[maschera_altro]  # IndexingError!

# ✓ CORRETTO: maschera dallo stesso DataFrame
maschera = df["Col"] > 0
df[maschera]
```

---

#### Errore 3: Confronto con NaN

```python
# ❌ SBAGLIATO: NaN == NaN restituisce False!
df[df["Col"] == np.nan]  # Non trova nulla

# ✓ CORRETTO: usa isna() o notna()
df[df["Col"].isna()]    # Righe con NaN
df[df["Col"].notna()]   # Righe senza NaN
```

---

#### Errore 4: Modifica su view (SettingWithCopyWarning)

```python
# ❌ Potenzialmente problematico
df_filtrato = df[df["A"] > 0]
df_filtrato["B"] = 100  # Warning!

# ✓ CORRETTO: copia esplicita
df_filtrato = df[df["A"] > 0].copy()
df_filtrato["B"] = 100  # OK
```

## 📌 Sezione 7 – Conclusione e Takeaways

### 🎓 Cosa abbiamo imparato in questa lezione

In questa lezione abbiamo costruito le fondamenta del **data wrangling** in Pandas:

1. **Boolean Indexing**: la tecnica centrale per filtrare DataFrame
   - Creare maschere booleane con operatori di confronto
   - Combinare condizioni con `&` (AND) e `|` (OR)
   - Sempre parentesi attorno alle singole condizioni!

2. **Accessor .loc[]**: selezione simultanea di righe e colonne
   - `df.loc[maschera, colonne]` per controllo granulare
   - Differenza tra lista (→DataFrame) e stringa (→Series)

3. **Filtri basati su statistiche**: approccio professionale
   - Usare `.quantile()` per soglie adattive
   - Pattern: calcola soglia → crea maschera → applica filtro

4. **Feature Engineering**: creare valore dai dati
   - Nuove colonne con operazioni aritmetiche
   - Flag binari con `.astype(int)`
   - Nomenclatura chiara e consistente

5. **Ordinamento**: ranking e presentazione
   - `.sort_values()` per ordinare
   - Parametro `ascending` per direzione
   - Ordinamento multi-colonna per gerarchie

---

### 🔗 Connessioni con le prossime lezioni

| Concetto di oggi | Approfondimento futuro |
|------------------|------------------------|
| Boolean Indexing | Query strings con `.query()` |
| Feature Engineering | Trasformazioni con `.apply()` e lambda |
| Ordinamento | Ranking con `.rank()` |
| Statistiche per soglie | GroupBy e aggregazioni |

---

### 📊 Checklist di autovalutazione

Prima di procedere alla prossima lezione, verifica di saper:

- [ ] Creare una maschera booleana da una condizione
- [ ] Combinare più condizioni con `&` e `|`
- [ ] Usare `.loc[]` per selezionare righe e colonne
- [ ] Calcolare quantili e usarli come soglie
- [ ] Creare nuove colonne con formule aritmetiche
- [ ] Creare flag binari da condizioni
- [ ] Ordinare un DataFrame per una o più colonne

### 🏁 Riepilogo comandi chiave

```python
# === FILTRAGGIO ===
df[df["Col"] > valore]                      # Filtro semplice
df[(cond1) & (cond2)]                       # AND di condizioni
df[(cond1) | (cond2)]                       # OR di condizioni
df[~condizione]                             # NOT (negazione)

# === SELEZIONE CON .loc[] ===
df.loc[maschera, ["Col1", "Col2"]]          # Righe filtrate, colonne specifiche
df.loc[maschera, "Col"]                     # Righe filtrate, singola colonna (Series)

# === QUANTILI ===
df["Col"].quantile(0.5)                     # Mediana
df["Col"].quantile(0.75)                    # Terzo quartile

# === FEATURE ENGINEERING ===
df["Nuova"] = df["A"] + df["B"]             # Operazione aritmetica
df["Flag"] = (df["A"] > soglia).astype(int) # Flag binario

# === ORDINAMENTO ===
df.sort_values(by="Col")                    # Ascendente
df.sort_values(by="Col", ascending=False)  # Discendente
df.sort_values(by=["A", "B"], ascending=[True, False])  # Multi-colonna
```

## 📌 Sezione 8 – Changelog e Metadata

### 📋 Changelog del notebook

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | Originale | Struttura base con esempi di filtraggio |
| 2.0 | Espansione | Aggiunta struttura 8 sezioni, teoria approfondita, micro-checkpoint, sanity check, glossario, debug reference |

---

### 📚 Riferimenti e risorse

- [Pandas Documentation - Indexing and Selecting Data](https://pandas.pydata.org/docs/user_guide/indexing.html)
- [Pandas Documentation - Boolean Indexing](https://pandas.pydata.org/docs/user_guide/indexing.html#boolean-indexing)
- [Real Python - Pandas DataFrame Filtering](https://realpython.com/pandas-dataframe/)

---

### 🔧 Requisiti tecnici

| Requisito | Versione minima |
|-----------|-----------------|
| Python | 3.8+ |
| pandas | 1.0+ |
| numpy | 1.19+ |

---

### 📝 Note per l'istruttore

- **Tempo stimato**: 45-60 minuti per esecuzione completa
- **Prerequisiti**: Lesson_01 (NumPy base, concetto di array/Series)
- **Punti critici**: 
  - Enfatizzare la differenza tra `and`/`or` e `&`/`|`
  - Far eseguire i micro-checkpoint per verificare comprensione
  - Lasciare tempo per gli esercizi della Sezione 5

---

### ✅ Completamento lezione

- [x] Sezione 1: Titolo e Obiettivi
- [x] Sezione 2: Teoria Approfondita  
- [x] Sezione 3: Modello Mentale
- [x] Sezione 4: Sezione Dimostrativa
- [x] Sezione 5: Esercizi Pratici
- [x] Sezione 6: Glossario e Debug
- [x] Sezione 7: Conclusione
- [x] Sezione 8: Changelog

**Status: LESSON COMPLETED ✓**